In [ ]:
import os,glob
import numpy as np
import cv2
import glob
import pickle
import tensorflow as tf
import argparse
import re
import datetime
from tensorflow.keras.layers import  Input,Conv2D,BatchNormalization,Activation,Subtract,LeakyReLU,Add,Average,Lambda,MaxPool2D,Dropout,UpSampling2D,Concatenate,Multiply,GlobalAveragePooling2D,Dense,ZeroPadding2D,AveragePooling2D
from tensorflow.keras.layers import concatenate,Flatten,ConvLSTM2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K
from sklearn.svm import LinearSVC
import matplotlib.pyplot as plt
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from skimage.feature import hog,local_binary_pattern
from skimage import data, exposure

In [ ]:
!git clone https://github.com/ieee8023/covid-chestxray-dataset.git

In [ ]:
!git clone https://github.com/kastnerkyle/deform-conv.git

In [ ]:
f = open("/content/deform-conv/deform_conv/layers.py","wb") 

In [ ]:
f

In [ ]:
from f import ConvOffset2D

In [ ]:
normal_dir = "/content/covid-chestxray-dataset/images"
dir1 = os.path.join(normal_dir,"*.png")
dir = os.path.join(normal_dir,"*.jpg")
dir2 = os.path.join(normal_dir,"*.jpeg")
covid_files = glob.glob(dir)
covid_files2 = glob.glob(dir2)
covid_files1 = glob.glob(dir1)
covid_files.extend(covid_files2)
covid_files.extend(covid_files1)

In [ ]:
covid_files[0]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

for i in range(len(covid_files)):
  img = cv2.imread(covid_files[i])
  img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
  img = img.astype('float32')/255.0
  im = np.asarray(img)
  
  plt.imshow(im)


In [ ]:
len(covid_files)

In [ ]:
 ! pip install -q kaggle

In [ ]:
from google.colab import files

files.upload()

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

In [ ]:

! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

In [ ]:
!unzip chest-xray-pneumonia.zip

In [ ]:
normal_dir = "/content/chest_xray/chest_xray/train/NORMAL"
dir1 = os.path.join(normal_dir,"*.png")
dir2 = os.path.join(normal_dir,"*.jpeg")
dir = os.path.join(normal_dir,"*.jpg")
normal_files = glob.glob(dir)
normal_1 = glob.glob(dir1)
normal_2 = glob.glob(dir2)
normal_files.extend(normal_1)
normal_files.extend(normal_2)

In [ ]:
len(normal_files)

In [ ]:
normal_dir = "/content/chest_xray/chest_xray/train/PNEUMONIA"
dir1 = os.path.join(normal_dir,"*.png")
dir2 = os.path.join(normal_dir,"*.jpeg")
dir = os.path.join(normal_dir,"*.jpg")
pneumonia_files = glob.glob(dir)
pneumonia_1 = glob.glob(dir1)
pneumonia_2 = glob.glob(dir2)
pneumonia_files.extend(pneumonia_1)
pneumonia_files.extend(pneumonia_2)

In [ ]:
len(pneumonia_files)

In [ ]:
train_dic = {}
for f in covid_files[:739]:
  train_dic[f] = [1,0,0]
for f in normal_files[:1072]:
  train_dic[f] = [0,1,0]
for f in pneumonia_files[:3100]:
  train_dic[f] = [0,0,1]

In [ ]:
len(pneumonia_files)

In [ ]:
test_dic = {}
for f in covid_files[739:]:
  test_dic[f] = [1,0,0]
for f in normal_files[1072:]:
  test_dic[f] = [0,1,0]
for f in pneumonia_files[3100:]:
  test_dic[f] = [0,0,1]

In [ ]:
dic = list(test_dic.keys())

In [ ]:
import random
l = list(train_dic.items())
random.shuffle(l)
l

In [ ]:

import random
l_test = list(test_dic.items())
random.shuffle(l_test)
l_test

In [ ]:
print(len(l),len(l_test))

In [ ]:
train_features = []
covid_dic_list = {}
data = []
labels = []
for i in range(len(l)):
  file_name,label = l[i]
  img = cv2.imread(file_name)
  try:
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    height, width = img.shape[:2]
    #for i in range(1,5):
      #rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2),15*i, 1)
      #image_rotated = cv2.warpAffine(img, rotation_matrix, (width, height))
    img = img.astype('float32')/255.0
    data.append(img)
    labels.append(label)

  except:
    print(i,file_name)
    print("Not possible")  
  
       
train_data = np.array(data)
print(train_data.shape)

train_labels = np.array(labels)
print(train_labels.shape)    
#data = data.reshape((data.shape[0]*data.shape[1],data.shape[2],data.shape[3],1))
#discard_n = len(data)-len(data)//batch_size*batch_size;
#data = np.delete(data,range(discard_n),axis = 0)
print('^_^-training data finished-^_^')

In [ ]:
test_features = []
data = []
labels = []
for i in range(len(l_test)):
  file_name,label = l_test[i]
  img = cv2.imread(file_name)
  try:
    img = cv2.resize(img,(224,224),interpolation = cv2.INTER_CUBIC)
    fd,hog_img = hog(img,orientations=8,pixels_per_cell=(16,16),cells_per_block=(1,1),visualize=True,multichannel=True)
    feature = hog_img.astype('float32')/255.0   
    img = img.astype('float32')/255.0
    test_features.append(feature)
    data.append(img)
    labels.append(label)
  except:
    print(file_name,i)  
 
  
test_data = np.array(data)
print(test_data.shape)

test_labels = np.array(labels)
print(test_labels.shape)    
#data = data.reshape((data.shape[0]*data.shape[1],data.shape[2],data.shape[3],1))
#discard_n = len(data)-len(data)//batch_size*batch_size;
#data = np.delete(data,range(discard_n),axis = 0)
print('^_^-testing data finished-^_^')

In [ ]:

inception_model = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),
                                               include_top=False,
                                               weights='imagenet',classes = 3)

In [ ]:
inputs = tf.keras.Input(shape=(224,224,3))
#  inputs_1 = tf.expand_dims(inputs,axis = 1)
#  lstm = tf.keras.layers.ConvLSTM2D(filters = 256,kernel_size = (3,3))(inputs_1)
x = inception_model(inputs)
input_ = tf.expand_dims(x,axis = 1)
nb_chan = 512
ratio = 16
x3 = ConvLSTM2D(filters=512, kernel_size=(1,1),padding = "same")(input_) 
x3 = BatchNormalization(axis=3, momentum=0.0,epsilon=0.0001)(x3)
x3 = Activation('relu')(x3)

#x_add = Add()([x1,x3])
y = tf.keras.layers.GlobalAveragePooling2D()(x3)
y = tf.keras.layers.Dense(nb_chan // ratio, activation='relu')(y)
y = tf.keras.layers.Dense(nb_chan, activation='sigmoid')(y)
y_3 = tf.keras.layers.Multiply()([x3, y])

ratio = 16



flat = Flatten()(y_3)

dense_1 = Dense(4096,activation = 'relu')(flat)
#dense_1 = Dropout(0.5)(dense_1)
dense_2 = Dense(4096,activation = 'relu')(dense_1)
#dense_2 = Dropout(0.5)(dense_2)
prediction = Dense(3,activation = 'softmax')(dense_2)

inception_pred = Model(inputs = inputs,outputs = prediction)

In [ ]:
inception_pred.summary()

In [ ]:
inception_pred.compile(optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.0002), loss=tf.keras.losses.CategoricalCrossentropy(from_logits = False) , metrics=['accuracy'])

In [ ]:
inception_pred.fit(train_data,train_labels,batch_size = 32,epochs = 60)

In [ ]:
inception_pred.save("mobilenet.h5")

In [ ]:
inception_pred = tf.keras.models.load_model("VGG19.h5",compile = False)

In [ ]:
inception_pred.evaluate(test_data,test_labels)

In [ ]:
x = inception_pred.predict(test_data)

In [ ]:
x_pred = np.argmax(x, axis=1) 

In [ ]:
for f in range(len(test_labels)):
  print(f,test_labels[f])

In [ ]:
x_pred[29]

In [ ]:
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:

img_size = (224, 224)
last_conv_layer_name = "vgg19"
classifier_layer_names = [
    "tf_op_layer_ExpandDims",
    "conv_lst_m2d",
    "batch_normalization",
    "activation",
    "global_average_pooling2d",
    "dense",
    "dense_1",
    "multiply",
    "flatten",                      
    "dense_2",                     
    "dense_3",
    "dense_4",
]

# The local path to our target image
plt.imshow(test_data[1137])

In [ ]:
240

In [ ]:
test_labels[1]

In [ ]:
x_pred[1]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display


from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [ ]:
classifier_input = keras.Input(shape=inception_model.output.shape[1:])
x = classifier_input

for layer_name in classifier_layer_names:
  print(layer_name)
  if layer_name == "activation":
    x = inception_pred.get_layer(layer_name)(x)
    k = x
  elif layer_name == "multiply":
    x = inception_pred.get_layer(layer_name)([x,k])
  else:  
    x = inception_pred.get_layer(layer_name)(x)
classifier_model = keras.Model(classifier_input, x)  
img_array = test_data[1117]
with tf.GradientTape() as tape:
  img = np.expand_dims(img_array,axis = 0)
  last_conv_layer_output = inception_model(img)
  tape.watch(last_conv_layer_output)
  preds = classifier_model(last_conv_layer_output)
  top_pred_index = tf.argmax(preds[0])
  top_class_channel = preds[:, top_pred_index]
grads = tape.gradient(top_class_channel, last_conv_layer_output)
pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
last_conv_layer_output = last_conv_layer_output.numpy()[0]
pooled_grads = pooled_grads.numpy()  
for i in range(pooled_grads.shape[-1]):
  last_conv_layer_output[:, :, i] *= pooled_grads[i]
heatmap = np.mean(last_conv_layer_output, axis=-1)
heatmap = np.maximum(heatmap, 0) / np.max(heatmap)

plt.imshow(heatmap)
plt.show()

In [ ]:

img = keras.preprocessing.image.img_to_array(img_array)

# We rescale heatmap to a range 0-255
heatmap = np.uint8(255 * heatmap)

# We use jet colormap to colorize heatmap
jet = cm.get_cmap("jet")

# We use RGB values of the colormap
jet_colors = jet(np.arange(256))[:, :3]
jet_heatmap = jet_colors[heatmap]

# We create an image with RGB colorized heatmap
jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

# Superimpose the heatmap on original image
superimposed_img = jet_heatmap * 0.0015 + img
superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)


In [ ]:
plt.imshow(test_data[980])
fig1 = plt.gcf()
plt.show()
fig1.savefig('mobilenet_covid_normal.png', dpi=300)

In [ ]:
plt.imshow(superimposed_img)
fig1 = plt.gcf()
plt.show()
fig1.savefig('mobilenet_covid_grad.png', dpi=300)

In [ ]:
plt.savefig("grad_cam_xray.jpg")

In [ ]:
test_labels[224]

In [ ]:
x_pred[224]